# Use Customer_Segmentation Notebook Template

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,AgglomerativeClustering,DBSCAN,SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

Matplotlib created a temporary cache directory at /tmp/matplotlib-49ms63jb because the default path (/home/mosaic-ai/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Read data from Snowflake CUSTOMER_TRANSACTION_MASTER using fosforio sdk

In [2]:
from fosforio import snowflake

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [3]:
# To get snowflake connection object with a default snowflake connection created by the user, if available.
#snowflake.get_connection()

# To get snowflake connection object with a specific connection name
snowflake.get_connection(connection_name="FDC_Insurance_Snowflake")

Exception occurred in getting snowflake connection: 'connectionSources'


In [ ]:
# To read a specific dataset published from a snowflake connection
df = snowflake.get_dataframe("CUSTOMER_TRANSACTION_MASTER")

In [ ]:
df_original = df.copy()

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
to_drop = ['CUSTOMER_ID','DATE','AGE','SEX','EDUCATION_LEVEL','EMPLOYMENT_STATUS','HOBBIES','MARITAL_STATUS','BANK_ACCOUNT_TYPE',
 'DEPENDENTS','REGISTRATION_DATE','PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY','BALANCE_FREQUENCY']

In [ ]:
df.drop(to_drop, axis= 1, inplace= True)

In [ ]:
df.info()

In [ ]:
scalar=StandardScaler()
scaled_df = scalar.fit_transform(df)

In [ ]:
scaled_df.shape

### PCA

In [ ]:
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_df)
pca_df = pd.DataFrame(data=principal_components ,columns=["PCA1","PCA2"])
pca_df

In [ ]:
k= range(2,6)
inertia_values= []
silhoutte_scores= []

for i in k:
    print(f'Trying KMean clusttering for k= {i}')
    kmean= KMeans(n_clusters= i, random_state= 42)
    kmean.fit(pca_df)
    inertia_values.append(kmean.inertia_)
    
    sil_score= silhouette_score(pca_df, kmean.labels_)
    silhoutte_scores.append(sil_score)

In [ ]:
with plt.style.context('dark_background'):
    plt.figure(figsize=(9,4))
    plt.title("Elbow Method")

    plt.subplot(1,2,1)
    plt.plot(k, inertia_values, marker= 'o', linestyle= '-')
    plt.xlabel("No of clusters (K)")
    plt.ylabel("Inertia")

    plt.subplot(1,2,2)
    plt.plot(k, silhoutte_scores, marker= 'o', linestyle= '-')
    plt.xlabel("No of clusters (K)")
    plt.ylabel("Silhouette Score")

    plt.show()

In [ ]:
k= 4
kmean= KMeans(n_clusters=k, random_state= 42)
kmean.fit(pca_df)

In [ ]:
pca_df['Group']= kmean.labels_

In [ ]:
pca_df.columns= ['PC1','PC2','Group']

In [ ]:
with plt.style.context('dark_background'):
    plt.figure(figsize=(10,8))
    sns.scatterplot(x="PC1",y="PC2",hue="Group",data=pca_df,palette='flare')
    plt.title("Groups Visualization")
    plt.show()

In [ ]:
pca_df['Group'].value_counts()

In [ ]:
cluster_centers = pd.DataFrame(data=kmean.cluster_centers_,columns=['PC1','PC2'])

In [ ]:
cluster_centers

In [ ]:
cluster_df = pd.concat([df_original,pd.DataFrame({'Cluster':kmean.labels_})],axis=1)

In [ ]:
cluster_df.head()

In [ ]:
cluster_1_df = cluster_df[cluster_df["Cluster"]==0]
cluster_1_df

In [ ]:
cluster_2_df = cluster_df[cluster_df["Cluster"]==1]
cluster_2_df

In [ ]:
cluster_3_df = cluster_df[cluster_df["Cluster"]==2]
cluster_3_df

In [ ]:
cluster_4_df = cluster_df[cluster_df["Cluster"] == 3]
cluster_4_df

In [ ]:
sns.countplot(x='Cluster', data=cluster_df)

In [44]:
cluster_df.to_csv("/data/Output/kmeans_clustered_customer_Data.csv")

In [45]:
import sys
sys.getsizeof(kmean)

48

In [46]:
#Saving Scikitlearn models
import joblib
joblib.dump(kmean, "/data/Output/kmeans_model.pkl")

['/data/Output/kmeans_model.pkl']